In [1]:
# Install dependencies if needed (TensorFlow and NumPy)
!pip install numpy tensorflow --quiet

# Import necessary libraries
import numpy as np
import tensorflow as tf
from scipy.sparse import coo_matrix

# 1. Simulate a Small Graph Dataset for Demonstration
def create_sample_graph():
    # Simulating a graph with 6 nodes and 8 edges
    adj_data = np.array([1, 1, 1, 1, 1, 1, 1, 1])
    adj_row = np.array([0, 1, 2, 3, 4, 0, 1, 2])
    adj_col = np.array([1, 2, 3, 4, 5, 2, 3, 4])

    # Adjacency matrix in sparse format
    n_nodes = 6
    adj_matrix = coo_matrix((adj_data, (adj_row, adj_col)), shape=(n_nodes, n_nodes))
    adj_matrix = adj_matrix + adj_matrix.T + coo_matrix(np.eye(n_nodes))  # Add self-loops

    # Node features: Random features for simplicity
    features = np.random.rand(n_nodes, 5)

    # Labels: Dummy labels (one-hot encoding for 3 classes)
    labels = np.array([
        [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1]
    ])
    return adj_matrix, features, labels

adj_matrix, features, labels = create_sample_graph()

# Normalize adjacency matrix
def normalize_adj(adj):
    rowsum = np.array(adj.sum(1)).flatten()
    d_inv_sqrt = np.power(rowsum, -0.5)
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.0
    d_mat_inv_sqrt = np.diag(d_inv_sqrt)
    return d_mat_inv_sqrt @ adj @ d_mat_inv_sqrt

normalized_adj = normalize_adj(adj_matrix.toarray())

# Convert adjacency to sparse tensor for TensorFlow
adj_tf = tf.sparse.from_dense(normalized_adj.astype(np.float32))  # Ensure float32 type


# 2. Define the Neural Network
def build_neural_network(input_dim, hidden_dim, output_dim):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(output_dim)
    ])

# Build the network
input_dim = features.shape[1]
hidden_dim = 64
output_dim = labels.shape[1]
model = build_neural_network(input_dim, hidden_dim, output_dim)

# 3. APPNP Propagation
def appnp_propagation(alpha, adj_matrix, features, num_iterations):
    Z = features  # Initialize with the input features
    for _ in range(num_iterations):
        Z = (1 - alpha) * tf.sparse.sparse_dense_matmul(adj_matrix, Z) + alpha * features
    return tf.nn.softmax(Z)

def train_model(features, labels, adj_matrix, alpha, num_iterations, learning_rate, epochs):
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    acc_fn = tf.keras.metrics.CategoricalAccuracy()

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            # Step 1: Predict
            predictions = model(features, training=True)

            # Step 2: Propagate
            propagated_predictions = appnp_propagation(alpha, adj_matrix, predictions, num_iterations)

            # Step 3: Compute loss
            loss = loss_fn(labels, propagated_predictions)

        # Backpropagation
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Compute accuracy
        acc_fn.update_state(labels, propagated_predictions)
        acc = acc_fn.result().numpy()
        acc_fn.reset_state()  # Corrected method to reset metrics

        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.numpy():.4f}, Accuracy: {acc:.4f}")

def evaluate_model(features, labels, adj_matrix, alpha, num_iterations):
    predictions = model(features, training=False)
    propagated_predictions = appnp_propagation(alpha, adj_matrix, predictions, num_iterations)
    accuracy = tf.reduce_mean(tf.keras.metrics.categorical_accuracy(labels, propagated_predictions))
    print(f"Test Accuracy: {accuracy.numpy():.4f}")

# Hyperparameters
alpha = 0.1
num_iterations = 10
learning_rate = 0.01
epochs = 50

# Convert features and labels to TensorFlow tensors
features_tf = tf.convert_to_tensor(features, dtype=tf.float32)
labels_tf = tf.convert_to_tensor(labels, dtype=tf.float32)

# Train the model
train_model(features_tf, labels_tf, adj_tf, alpha, num_iterations, learning_rate, epochs)

# Evaluate the model
evaluate_model(features_tf, labels_tf, adj_tf, alpha, num_iterations)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50 - Loss: 1.1162, Accuracy: 0.3333
Epoch 2/50 - Loss: 1.1086, Accuracy: 0.1667
Epoch 3/50 - Loss: 1.1134, Accuracy: 0.1667
Epoch 4/50 - Loss: 1.1053, Accuracy: 0.3333
Epoch 5/50 - Loss: 1.1026, Accuracy: 0.3333
Epoch 6/50 - Loss: 1.0902, Accuracy: 0.3333
Epoch 7/50 - Loss: 1.1239, Accuracy: 0.3333
Epoch 8/50 - Loss: 1.1159, Accuracy: 0.3333
Epoch 9/50 - Loss: 1.0728, Accuracy: 0.3333
Epoch 10/50 - Loss: 1.0963, Accuracy: 0.3333
Epoch 11/50 - Loss: 1.0907, Accuracy: 0.3333
Epoch 12/50 - Loss: 1.0828, Accuracy: 0.3333
Epoch 13/50 - Loss: 1.0742, Accuracy: 0.3333
Epoch 14/50 - Loss: 1.0682, Accuracy: 0.3333
Epoch 15/50 - Loss: 1.0867, Accuracy: 0.3333
Epoch 16/50 - Loss: 1.0807, Accuracy: 0.3333
Epoch 17/50 - Loss: 1.0891, Accuracy: 0.3333
Epoch 18/50 - Loss: 1.1183, Accuracy: 0.3333
Epoch 19/50 - Loss: 1.0761, Accuracy: 0.6667
Epoch 20/50 - Loss: 1.1010, Accuracy: 0.5000
Epoch 21/50 - Loss: 1.0575, Accuracy: 0.5000
Epoch 22/50 - Loss: 1.0691, Accuracy: 0.3333
Epoch 23/50 - Loss: